In [1]:
import torch
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torch import nn
from PIL import Image
import pandas as pd
import os
import torchvision.io
from torchvision.transforms import ToTensor
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import f1_score

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Get cpu, gpu or mps device for training.
DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {DEVICE} device")

NUM_WORKERS = 0

Using cuda device


In [ ]:
def load_dataset(dataset):
    xs = []
    ys = []
    for x, y in dataset:
        xs += [x.to(DEVICE)]
        ys += [torch.tensor(y).view(-1,1).to(DEVICE)]
    xs = torch.cat(xs)
    ys = torch.cat(ys).view(-1,)
    return TensorDataset(xs, ys)


class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 1])
        image = read_image(img_path,ImageReadMode.RGB).float()
       
        label = self.img_labels.iloc[idx, 4]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
class Data:
    def __init__(self, batch_size,dataset_train,dataset_test):
        self.batch_size = batch_size
        self.training_data = load_dataset(dataset_train)
        self.test_data = load_dataset(dataset_test)
    
    def get_loader(self, training: bool):
        if training:
            dataloader = DataLoader(self.training_data,batch_size=self.batch_size, shuffle=True)
        else:
            dataloader = DataLoader(self.test_data,batch_size=self.batch_size, shuffle=False)
        return dataloader
'''    
class Data:
    def __init__(self):
        root = 'data'
        batch_size = 64
        self.datasets = {}
        self.dataloaders = {}
        for is_train in [True, False]:
            ds = datasets.FashionMNIST(
                root=root,
                train=is_train,
                download=True,
                transform=ToTensor(),
            )
            self.datasets[is_train] = load_dataset(ds)

            self.dataloaders[is_train] = DataLoader(
                self.datasets[is_train],
                batch_size,
                shuffle=is_train,
                num_workers=NUM_WORKERS,
            )

    def get_loader(self, is_train: bool):
        return self.dataloaders[is_train]
'''

In [ ]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

class Learner:
    def __init__(self):
        self.model = NeuralNetwork()
        self.model.to(DEVICE)
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=1e-3)

    def predict(self, x):
        return self.model(x)

    def update(self, loss):
        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
class Evaluator:
    def __init__(self):
        self.loss_fn = nn.CrossEntropyLoss()

    def get_loss(self, y, y_hat):
        return self.loss_fn(y_hat, y)

In [ ]:
import time

class Trainer:
    def __init__(self, data: Data, learner: Learner, evaluator: Evaluator):
        self.data = data
        self.learner = learner
        self.evaluator = evaluator

    def one_epoch(self, training: bool):
        self.learner.model.train(training)
        dataloader = self.data.get_loader(training)
        test_loss, correct = 0, 0
        num_batches = len(dataloader)
        size = len(dataloader.dataset)
        for batch_idx, (X, y) in enumerate(dataloader):
            #X, y = X.to(DEVICE), y.to(DEVICE)
            y_hat = self.learner.predict(X)
            loss = self.evaluator.get_loss(y, y_hat)
            if training:
                self.learner.update(loss)
                if batch_idx % 100 == 0:
                    loss, current = loss.item(), (batch_idx + 1) * len(X)
                    print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            else:
                test_loss += loss.item()
                correct += (y_hat.argmax(1) == y).type(torch.float).sum().item()
        if not training:
            test_loss /= num_batches
            correct /= size
            print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    def run(self, n_epochs: int):
        for t in range(n_epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            #start = time.time()
            self.one_epoch(training=True)
            #end = time.time()
            #print(f"time: {end - start:.2f}s")
            with torch.no_grad():
                self.one_epoch(training=False)
        print("Done!")

In [ ]:
data = Data()

In [ ]:
for X, y in data.get_loader(False):
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [ ]:
learner = Learner()

In [ ]:
evaluator = Evaluator()

In [ ]:
trainer = Trainer(data, learner, evaluator)

In [ ]:
trainer.run(25)

Epoch 1
-------------------------------
Test Error: 
 Accuracy: 66.1%, Avg loss: 0.959632 

Epoch 2
-------------------------------
Test Error: 
 Accuracy: 67.3%, Avg loss: 0.889571 

Epoch 3
-------------------------------
Test Error: 
 Accuracy: 68.6%, Avg loss: 0.839861 

Epoch 4
-------------------------------
Test Error: 
 Accuracy: 69.4%, Avg loss: 0.803905 

Epoch 5
-------------------------------
Test Error: 
 Accuracy: 71.7%, Avg loss: 0.773915 

Epoch 6
-------------------------------
Test Error: 
 Accuracy: 72.6%, Avg loss: 0.747857 

Epoch 7
-------------------------------
Test Error: 
 Accuracy: 73.8%, Avg loss: 0.725759 

Epoch 8
-------------------------------
Test Error: 
 Accuracy: 74.7%, Avg loss: 0.706623 

Epoch 9
-------------------------------
Test Error: 
 Accuracy: 75.2%, Avg loss: 0.690545 

Epoch 10
-------------------------------
Test Error: 
 Accuracy: 76.3%, Avg loss: 0.673279 

Epoch 11
-------------------------------
Test Error: 
 Accuracy: 76.7%, Avg los